In [17]:
!pip install langdetect

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993363 sha256=e5ab45865e663966d39442000a929ad86145c1f9100b225d86ffa35fa805bf89
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\eb\87\25\2dddf1c94e1786054e25022ec5530bfed52bad86d882999c48
Successfully built langdetect


In [1]:
import pandas as pd

shows = pd.read_csv('shows_with_genre_list.csv')

In [5]:
len(shows[shows['vote_count']==0])

108290

In [ ]:
shows

In [2]:
from transformers import pipeline 
classifier = pipeline('text-classification',
                      model='j-hartmann/emotion-english-distilroberta-base',
                      top_k = None,
                      device=0)
classifier("I love this!")

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528676815330982},
  {'label': 'neutral', 'score': 0.005764591973274946},
  {'label': 'anger', 'score': 0.004419781267642975},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [3]:
classifier("my bad")

[[{'label': 'sadness', 'score': 0.898618757724762},
  {'label': 'surprise', 'score': 0.027377739548683167},
  {'label': 'disgust', 'score': 0.023179685696959496},
  {'label': 'neutral', 'score': 0.020622100681066513},
  {'label': 'fear', 'score': 0.01389271765947342},
  {'label': 'joy', 'score': 0.008949924260377884},
  {'label': 'anger', 'score': 0.007359103765338659}]]

In [4]:
shows['overview'][0]

"Seven noble families fight for control of the mythical land of Westeros. Friction between the houses leads to full-scale war. All while a very ancient evil awakens in the farthest north. Amidst the war, a neglected military order of misfits, the Night's Watch, is all that stands between the realms of men and icy horrors beyond."

In [5]:
classifier(shows['overview'][0])

[[{'label': 'fear', 'score': 0.6370058655738831},
  {'label': 'disgust', 'score': 0.1523459404706955},
  {'label': 'anger', 'score': 0.08531393855810165},
  {'label': 'neutral', 'score': 0.07609027624130249},
  {'label': 'sadness', 'score': 0.04325457289814949},
  {'label': 'surprise', 'score': 0.003916653338819742},
  {'label': 'joy', 'score': 0.002072726609185338}]]

In [6]:
classifier(shows['overview'][0].split("."))

[[{'label': 'neutral', 'score': 0.5428494215011597},
  {'label': 'anger', 'score': 0.31260135769844055},
  {'label': 'joy', 'score': 0.06551150977611542},
  {'label': 'fear', 'score': 0.03280727192759514},
  {'label': 'disgust', 'score': 0.017563313245773315},
  {'label': 'surprise', 'score': 0.01438913680613041},
  {'label': 'sadness', 'score': 0.014277953654527664}],
 [{'label': 'fear', 'score': 0.5626480579376221},
  {'label': 'anger', 'score': 0.30526384711265564},
  {'label': 'neutral', 'score': 0.07093406468629837},
  {'label': 'disgust', 'score': 0.029906222596764565},
  {'label': 'sadness', 'score': 0.015349512919783592},
  {'label': 'surprise', 'score': 0.013048793189227581},
  {'label': 'joy', 'score': 0.0028494831640273333}],
 [{'label': 'fear', 'score': 0.5953421592712402},
  {'label': 'disgust', 'score': 0.16899792850017548},
  {'label': 'neutral', 'score': 0.11854366958141327},
  {'label': 'anger', 'score': 0.04659063369035721},
  {'label': 'sadness', 'score': 0.044568661

In [7]:
sentences = shows['overview'][0].split(".")
predictions = classifier(sentences)

In [8]:
sentences[0]

'Seven noble families fight for control of the mythical land of Westeros'

In [9]:
predictions[0]

[{'label': 'neutral', 'score': 0.5428494215011597},
 {'label': 'anger', 'score': 0.31260135769844055},
 {'label': 'joy', 'score': 0.06551150977611542},
 {'label': 'fear', 'score': 0.03280727192759514},
 {'label': 'disgust', 'score': 0.017563313245773315},
 {'label': 'surprise', 'score': 0.01438913680613041},
 {'label': 'sadness', 'score': 0.014277953654527664}]

In [10]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.31260135769844055},
 {'label': 'disgust', 'score': 0.017563313245773315},
 {'label': 'fear', 'score': 0.03280727192759514},
 {'label': 'joy', 'score': 0.06551150977611542},
 {'label': 'neutral', 'score': 0.5428494215011597},
 {'label': 'sadness', 'score': 0.014277953654527664},
 {'label': 'surprise', 'score': 0.01438913680613041}]

In [11]:
import numpy as np
from tqdm import tqdm

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']

# Store results
id = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x:x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])

    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [12]:
for i in range(3):
    id.append(shows['id'][0])
    sentences = shows['overview'][0].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [13]:
emotion_scores

{'anger': [0.31260135769844055, 0.31260135769844055, 0.31260135769844055],
 'disgust': [0.16899792850017548, 0.16899792850017548, 0.16899792850017548],
 'fear': [0.711215078830719, 0.711215078830719, 0.711215078830719],
 'joy': [0.06551150977611542, 0.06551150977611542, 0.06551150977611542],
 'sadness': [0.5494765043258667, 0.5494765043258667, 0.5494765043258667],
 'surprise': [0.1116902157664299, 0.1116902157664299, 0.1116902157664299],
 'neutral': [0.07876554876565933, 0.07876554876565933, 0.07876554876565933]}

In [22]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']

# Store results
id = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(shows))):
    overview_text = shows['overview'][i]

    try:
        if detect(overview_text) != "en":
            continue
    except LangDetectException:
        continue

    id.append(shows["id"][i])
    sentences = shows['overview'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 164647/164647 [44:39<00:00, 61.45it/s] 


In [23]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["id"] = id

In [24]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,id
0,0.312601,0.168998,0.711215,0.065512,0.549477,0.111690,0.078766,1399
1,0.151468,0.104007,0.450262,0.040564,0.549477,0.111690,0.193565,71446
2,0.064134,0.104007,0.939563,0.040564,0.549477,0.111690,0.078766,66732
3,0.064134,0.249498,0.418831,0.053272,0.790493,0.111690,0.078766,1402
4,0.064134,0.480019,0.384543,0.040564,0.549477,0.947755,0.078766,63174
...,...,...,...,...,...,...,...,...
162297,0.064134,0.104007,0.051363,0.040564,0.822729,0.111690,0.078766,239099
162298,0.064134,0.104007,0.051363,0.040564,0.743006,0.180555,0.078766,241205
162299,0.064134,0.104007,0.867593,0.040564,0.948849,0.111690,0.078766,240696
162300,0.064294,0.545137,0.266021,0.040564,0.549477,0.111690,0.078766,240697


In [25]:
shows = pd.merge(shows, emotions_df, on='id')

In [26]:
shows

,id,name,number_of_seasons,number_of_episodes,original_language,vote_count,vote_average,overview,backdrop_path,first_air_date,...,genre_list,title_and_tagline,tagged_description,anger,disgust,fear,joy,sadness,surprise,neutral
0,1399,Game of Thrones,8,73,en,21857,8.442,Seven noble families fight for control of the ...,/2OMB0ynKlyIenMJWI2Dy9IWT4c.jpg,2011-04-17,...,"['Sci-Fi & Fantasy', 'Drama', 'Action & Advent...",Game of Thrones: Winter Is Coming,1399 Seven noble families fight for control of...,0.312601,0.168998,0.711215,0.065512,0.549477,0.111690,0.078766
1,71446,Money Heist,3,41,es,17836,8.257,"To carry out the biggest heist in history, a m...",/gFZriCkpJYsApPZEF3jhxL4yLzG.jpg,2017-05-02,...,"['Crime', 'Drama']",Money Heist: The perfect robbery.,71446 To carry out the biggest heist in histor...,0.151468,0.104007,0.450262,0.040564,0.549477,0.111690,0.193565
2,66732,Stranger Things,4,34,en,16161,8.624,"When a young boy vanishes, a small town uncove...",/2MaumbgBlW1NoPo3ZJO38A6v7OS.jpg,2016-07-15,...,"['Drama', 'Sci-Fi & Fantasy', 'Mystery']",Stranger Things: Every ending has a beginning.,"66732 When a young boy vanishes, a small town ...",0.064134,0.104007,0.939563,0.040564,0.549477,0.111690,0.078766
3,1402,The Walking Dead,11,177,en,15432,8.121,Sheriff's deputy Rick Grimes awakens from a co...,/x4salpjB11umlUOltfNvSSrjSXm.jpg,2010-10-31,...,"['Action & Adventure', 'Drama', 'Sci-Fi & Fant...",The Walking Dead: Fight the dead. Fear the liv...,1402 Sheriff's deputy Rick Grimes awakens from...,0.064134,0.249498,0.418831,0.053272,0.790493,0.111690,0.078766
4,63174,Lucifer,6,93,en,13870,8.486,"Bored and unhappy as the Lord of Hell, Lucifer...",/aDBRtunw49UF4XmqfyNuD9nlYIu.jpg,2016-01-25,...,"['Crime', 'Sci-Fi & Fantasy']",Lucifer: It's good to be bad.,"63174 Bored and unhappy as the Lord of Hell, L...",0.064134,0.480019,0.384543,0.040564,0.549477,0.947755,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173894,240697,女子大生危険なアルバイト,1,1,en,0,0.000,A college student gets into trouble when she m...,NaN,1990-02-10,...,"['Action & Adventure', 'Drama']",女子大生危険なアルバイト,240697 A college student gets into trouble whe...,0.064294,0.545137,0.266021,0.040564,0.549477,0.111690,0.078766
173895,237566,Welcome to My World,1,5,en,0,0.000,Short-lives series on Youtube Red.,NaN,2016-01-01,...,['Sci-Fi & Fantasy'],Welcome to My World,237566 Short-lives series on Youtube Red.,0.064134,0.104007,0.051363,0.040564,0.874031,0.111690,0.079212
173896,237566,Welcome to My World,1,5,en,0,0.000,Short-lives series on Youtube Red.,NaN,2016-01-01,...,['Sci-Fi & Fantasy'],Welcome to My World,237566 Short-lives series on Youtube Red.,0.064134,0.104007,0.051363,0.040564,0.874031,0.111690,0.079212
173897,237566,Welcome to My World,1,5,en,0,0.000,Short-lives series on Youtube Red.,NaN,2016-01-01,...,['Sci-Fi & Fantasy'],Welcome to My World,237566 Short-lives series on Youtube Red.,0.064134,0.104007,0.051363,0.040564,0.874031,0.111690,0.079212


In [27]:
shows.to_csv("shows_with_emotions.csv", index=False)